In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import matplotlib.pyplot as plt
import scorecardpy as sc

working_data = # post-data prep dataframe here, i.e. merged_data or something
train, test = sc.split_df(working_data, y = 'TARGET', ratio = .7).values()
card = # YOUR SCORECARD AFTER LR HERE
# something like this: sc.scorecard(bins, lr, X_train.columns, points0 = 600, odds0 = 1/19, pdo = 20, basepoints_eq0 = True)
train_score = sc.scorecard_ply(train, card)
test_score = sc.scorecard_ply(test, card)

def get_default_prob(cutoff, base_odds=20, pdo=20, base_score=600):
    odds_at_cutoff = base_odds / 2 ** ((base_score-cutoff)/pdo)
    default_prob = 1/(1+odds_at_cutoff)
    return default_prob

avg_loan_amount = 41000
avg_loss_per_default = 28700
cutoff_scores = np.arange(555, 600, 1)
profits_dict = {}

for effective_interest_rate in range(4, 20, 2):
    profits = []
    for cutoff in cutoff_scores:
        approval_count = train_score[train_score["score"]>cutoff].count()['score']
        default_count = (get_default_prob(cutoff) * approval_count).round()
        revenue = (approval_count - default_count) * avg_loan_amount * effective_interest_rate * 0.01
        loss = default_count * avg_loss_per_default
        profit = revenue - loss
        profits.append(profit)
    profits_dict[effective_interest_rate] = profits
    
plt.figure(figsize=(10,6))
for effective_interest_rate, profits in profits_dict.items():
    plt.plot(cutoff_scores, profits, marker='o', label=f'Effective Interest Rate = {effective_interest_rate}%')

plt.title("Profitability vs. Cutoff Score for Different EIRs")
plt.xlabel("Cutoff Score")
plt.ylabel("Profit (in millions)")
plt.grid(True)
plt.legend()
plt.show()

# Find the best cutoff score for each revenue per loan value
for revenue_per_loan, profits in profits_dict.items():
    best_idx = np.argmax(profits)
    print(f"Best cutoff for EIR {revenue_per_loan}%: {cutoff_scores[best_idx]}, Profit = ${profits[best_idx]:,.2f}")

